In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import librosa
import time

#transcriber = pipeline("automatic-speech-recognition", model="asapp/sew-d-tiny-100k-ft-ls100h")
transcriber = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
candidate_labels = ["Sound of a hand hitting a desk", "Sound of a clap", "Sound of random noise"]

def transcribe(audio):
    start=time.time()
    sr, y = audio
    y = y.astype(np.float32)
    if len(y.shape)>=2: y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    num_seconds = 5
    num_samples = len(y)
    last_five_seconds_start_index = num_samples - (num_seconds * sr)
    last_five_seconds_end_index = num_samples
    last_five_seconds = []
    for i in range(last_five_seconds_start_index, last_five_seconds_end_index): 
        last_five_seconds.append(y[i])
    arraylastfive = np.array(last_five_seconds)
    return transcriber(arraylastfive, candidate_labels=candidate_labels)


demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=True)


c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://694155c14481e32777.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\gradio\route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\gradio\blocks.py", line 1608, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\gradio\blocks.py", line 1176, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\shunc\anaconda3\envs\aim\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker

In [2]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(stream, new_chunk):
    """sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]"""
    sr, y = new_chunk
    if len(y.shape) > 1: y = np.mean(y,axis=1)
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    #librosa.resample(y, orig_sr=sr, target_sr=16000)
    return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]


demo = gr.Interface(
    transcribe,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e2222ba2c39582c1e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
